# LIBRARIES

In [1]:
import os
import ast
import json
import math
import ccxt
import time
import pickle
import random
import numpy as np
import configparser
import pandas as pd
import google.cloud
import tweepy as tw
from pandas_gbq import gbq
import matplotlib.cm as cm
from boruta import BorutaPy
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from google.cloud import bigquery
from sklearn import preprocessing
from requests import Request, Session
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix
from scipy.interpolate import UnivariateSpline
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

#_____

pd.set_option('display.max_rows', 5000)
pd.options.mode.chained_assignment = None

# FUNCTIONS

In [2]:
#FUNCION QUE LEE EL ARCHIVO CONFIG
def get_config(category, key):
    
    global config
    return config[category][key]

#_____

#FUNCION QUE GENERA CONEXION CON ARCHIVO CONFIG
def updateConfig(config_name):
    
    global config
    
    config = configparser.ConfigParser()
    config.sections()
    config.read(config_name)
    
#_____

#FUNCION QUE GRAFICA POR COLORES SEGUN COMPRAS Y NO COMRPAS
def plot_colourline(x,y,c):
    c = c
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=c[i])
    return

#_____

#FUNCION QUE ESCRIBE PICKLE DE VARIABLE
def writePickleVariable(variable,variable_name):
    pickle_out = open(variable_name+".pickle","wb")
    pickle.dump(variable, pickle_out)
    pickle_out.close()
    
#_____
    
#FUNCION QUE LEE PICKLE DE VARIABLE
def readPickleVariable(variable_name):    
    return pickle.load(open(variable_name+".pickle","rb"))

#_____

#FUNCION QUE REDONDEA FLOAT HACIA ABAJO SEGUN DECIMALES
def round_decimals_down(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

#_____

#FUNCION QUE REDONDEA FLOAT HACIA ARRIBA SEGUN DECIMALES
def round_decimals_up(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.ceil(number)

    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

#_____

#FUNCION QUE DESCARGA BASE DE DATOS DE MERCADO DE BIGQUERY
def downloadDataBaseBigQuery(market_model,time,rows=None):
    
    #_____GOOGLE CLOUD CONECTION
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/PREDICTION/bigQueryAccess.json"

    #_____SI NO SE DA UN NUMERO ESPECIFICO DE FILAS POR PARAMETRO
    if rows==None:

        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"] ORDER BY TIME DESC",project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    else:

        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"] ORDER BY TIME DESC LIMIT "+str(rows),project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    candlesDataBase_BigQuery.OPEN=candlesDataBase_BigQuery.OPEN.astype(float)
    candlesDataBase_BigQuery.HIGH=candlesDataBase_BigQuery.HIGH.astype(float)
    candlesDataBase_BigQuery.LOW=candlesDataBase_BigQuery.LOW.astype(float)
    candlesDataBase_BigQuery.CLOSE=candlesDataBase_BigQuery.CLOSE.astype(float)
    candlesDataBase_BigQuery.VOLUME=candlesDataBase_BigQuery.VOLUME.astype(float)

    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    if time != "1":

        #_____UPDATE CONFIG PREDICTION
        updateConfig("semiManual.ini")

        #_____CREAR COLUMNAS
        candlesDataBase_BigQuery_copy=candlesDataBase_BigQuery.copy()
        candlesDataBase_BigQuery_copy=candlesDataBase_BigQuery_copy.set_index('TIME')
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy[["ID"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).min()
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["VOLUME"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).sum())
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["LOW"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).min())
        candlesDataBase_BigQuery_copy_total=candlesDataBase_BigQuery_copy_total.join(candlesDataBase_BigQuery_copy[["HIGH"]].groupby(pd.Grouper(freq=time+'min', offset=time+'min')).max())

        #_____AGREAR DATOS
        for i in list(candlesDataBase_BigQuery_copy_total.index.values):
            try:
                candlesDataBase_BigQuery_copy_total.at[i,"OPEN"]=candlesDataBase_BigQuery.loc[candlesDataBase_BigQuery.TIME==i]["OPEN"].values[0]
                final_time = i + np.timedelta64(int(time),'m')
                if final_time in list(candlesDataBase_BigQuery_copy_total.index.values):
                    candlesDataBase_BigQuery_copy_total.at[i,"CLOSE"]=candlesDataBase_BigQuery.loc[candlesDataBase_BigQuery.TIME==final_time]["OPEN"].values[0]
                else:
                    candlesDataBase_BigQuery_copy_total.at[i,"CLOSE"]=candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"CLOSE"]
            except:
                pass

        #_____RESET INDEX
        candlesDataBase_BigQuery_copy_total.reset_index(level=['TIME'],inplace=True)

        #_____COLUMNA MARKET
        for i in range(0,len(candlesDataBase_BigQuery_copy_total)):
            candlesDataBase_BigQuery_copy_total.at[i,"MARKET"]=candlesDataBase_BigQuery.MARKET.values[0]

        #_____ORDENAR COLUMNAS
        column_names = list(candlesDataBase_BigQuery.columns)
        candlesDataBase_BigQuery_copy_total = candlesDataBase_BigQuery_copy_total.reindex(columns=column_names)
        candlesDataBase_BigQuery_copy_total.dropna(inplace=True)
        candlesDataBase_BigQuery_copy_total.reset_index(inplace=True,drop=True)
        candlesDataBase_BigQuery=candlesDataBase_BigQuery_copy_total.copy()

    #_____RETURN
    return candlesDataBase_BigQuery

#_____

#FUNCION QUE DEVUELVE EL RSI DADO UN PERIODO DETERMINADO
def RSI(candlesDataBase_BigQuery):
    
    global config
    updateConfig("semiManual.ini")
    
    series=pd.Series(list(candlesDataBase_BigQuery.CLOSE.values))
    period=int(get_config("PARAMETERS","RSI_LEN"))
    
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    
    rsi=100 - 100 / (1 + rs)
    
    for i in list(rsi.index.values):
        candlesDataBase_BigQuery.at[i,"RSI"]=rsi[i]
                                        
    return candlesDataBase_BigQuery

#_____

#FUNCION QUE CALCULA MACD
def MACD(candlesDataBase_BigQuery):
    
    global config
    updateConfig("semiManual.ini")
    
    close_data_df=candlesDataBase_BigQuery.copy()
    close_data_df.CLOSE=close_data_df.CLOSE.astype(float)
    close_data_df=close_data_df[["CLOSE"]]
    
    #_____CREAR VARIABLE MACD
    macd_12 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_SHORT")), adjust=False).mean()
    macd_26 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_LONG")), adjust=False).mean()
    macd = macd_12 - macd_26
    macd_9 = macd.ewm(span=int(get_config("PARAMETERS","MACD_TRIGGER")), adjust=False).mean()

    #_____AGREGAR VARIABLES A LA BASE DE DATOS
    candlesDataBase_BigQuery["MACD"]=macd
    #candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_SHORT")]=macd_12
    #candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_LONG")]=macd_26
    candlesDataBase_BigQuery["MACD_TRIGGER"]=macd_9
    
    return candlesDataBase_BigQuery

#_____

#FUNCION QUE BRINDA LOS NIVELES DE FIBONACCI
def fibonacciLevels(candlesDataBase_BigQuery):
    
    global config
    global fractal
    
    #_____UPDATE CONFIG PREDICTION
    updateConfig("semiManual.ini")
    
    #_____ACTUALIZAR PARAMETROS
    FIBONACCI_PERIOD=int(int(get_config("PARAMETERS","FIBONACCI_PERIOD"))/int(fractal))

    #_____RECORRER DATOS DE TEST
    for i in range(FIBONACCI_PERIOD,len(candlesDataBase_BigQuery)):
        
        #_____FIBONACCI
        lows=list(candlesDataBase_BigQuery[i-FIBONACCI_PERIOD:i].LOW.values)
        highs=list(candlesDataBase_BigQuery[i-FIBONACCI_PERIOD:i].HIGH.values)
        Price_Min = min(lows)
        Price_Max = max(highs)
        Diff = Price_Max-Price_Min
        level1 = Price_Max - 0.236 * Diff
        level2 = Price_Max - 0.382 * Diff
        level3 = Price_Max - 0.618 * Diff
        level4 = Price_Max - 0.786 * Diff
        
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_1"]=level1
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_2"]=level2
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_3"]=level3
        candlesDataBase_BigQuery.at[i,"FIBO_LEVEL_4"]=level4
            
    return candlesDataBase_BigQuery

#_____

#FUNCION QUE CALCULA MACD
def LAGS(candlesDataBase_BigQuery):
    
    for j in list(candlesDataBase_BigQuery.columns)[3:]:
        candlesDataBase_BigQuery[j+"_1"]=candlesDataBase_BigQuery[j].shift(1)
        candlesDataBase_BigQuery[j+"_2"]=candlesDataBase_BigQuery[j].shift(2)
        candlesDataBase_BigQuery[j+"_3"]=candlesDataBase_BigQuery[j].shift(3)
        
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
        
    return candlesDataBase_BigQuery

#_____

#CREATE VARIABLES
def variableCreation(candlesDataBase_BigQuery):
    
    global fractal
    updateConfig("semiManual.ini")
    
    candlesDataBase_BigQuery=RSI(candlesDataBase_BigQuery)
    candlesDataBase_BigQuery=MACD(candlesDataBase_BigQuery)
    #candlesDataBase_BigQuery=fibonacciLevels(candlesDataBase_BigQuery)
    #candlesDataBase_BigQuery=LAGS(candlesDataBase_BigQuery)
    
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    return candlesDataBase_BigQuery

#_____

#FUNCION QUE HACE BACKTESTING PARA DETERMINAR MEJORES PARAMETROS TOPES DE VENTA
def backtesting(test_sample):
    
    global fractal
    global final_df
    global utility_df
    global train_model
    global tradingBook
    global market_model
    global train_sample
    global backtestingDataFrame
    global distance
    global angle
    global rsi_15
    global rsi_30
    global rsi_60
    global takeprofit
    global stoploss
    
    #_____UPDATE CONFIG PREDICTION
    updateConfig("semiManual.ini")
    
    #_____INICIAR LISTAS
    TP_LIST=ast.literal_eval(get_config("PARAMETERS_BACKTESTING","TP_LIST"))
    SL_LIST=ast.literal_eval(get_config("PARAMETERS_BACKTESTING","SL_LIST"))
    
    PRECENTIL_DISTANCE_MACD_TRIGGER=ast.literal_eval(get_config("PARAMETERS_BACKTESTING","PRECENTIL_DISTANCE_MACD_TRIGGER"))
    PRECENTIL_ANGLE_MACD_TRIGGER=ast.literal_eval(get_config("PARAMETERS_BACKTESTING","PRECENTIL_ANGLE_MACD_TRIGGER"))
    RSI_15=ast.literal_eval(get_config("PARAMETERS_BACKTESTING","RSI_15"))
    RSI_30=ast.literal_eval(get_config("PARAMETERS_BACKTESTING","RSI_30"))
    RSI_60=ast.literal_eval(get_config("PARAMETERS_BACKTESTING","RSI_60"))
    
    #_____INICIALIZAR BASE DE DATOS DE SIMULACION DE UTILIDAD
    utility_df=pd.DataFrame()
    
    #_____FOR LOOP
    for distance in PRECENTIL_DISTANCE_MACD_TRIGGER:
        for angle in PRECENTIL_ANGLE_MACD_TRIGGER:
            for rsi_15 in RSI_15:
                for rsi_30 in RSI_30:
                    for rsi_60 in RSI_60:
                        for takeprofit in TP_LIST:
                            for stoploss in SL_LIST:
            
                                #_____GENERAR VARIABLES DE DECISION
                                candlesDataBase_BigQuery_Signal_COPY=test_sample.copy()

                                #

                                #CUANDO DISTANCIA ENTRE MACD Y TRIGGER ESTE EN PERCENTIL 10
                                candlesDataBase_BigQuery_Signal_COPY=candlesDataBase_BigQuery_Signal_COPY.loc[(candlesDataBase_BigQuery_Signal_COPY.HIST_MACD_VS_TRIGGER_15<=np.quantile(list(candlesDataBase_BigQuery_Signal_COPY.HIST_MACD_VS_TRIGGER_15.values),float(distance[1])))&(candlesDataBase_BigQuery_Signal_COPY.HIST_MACD_VS_TRIGGER_15>=np.quantile(list(candlesDataBase_BigQuery_Signal_COPY.HIST_MACD_VS_TRIGGER_15.values),float(distance[0])))]
                                candlesDataBase_BigQuery_Signal_COPY.reset_index(inplace=True,drop=True)

                                #ANGULO 15 ESTE SOBRE PERCENTIL 90
                                candlesDataBase_BigQuery_Signal_COPY=candlesDataBase_BigQuery_Signal_COPY.loc[candlesDataBase_BigQuery_Signal_COPY.ANGLE_MACD_VS_TRIGGER_15>=np.quantile(list(candlesDataBase_BigQuery_Signal_COPY["ANGLE_MACD_VS_TRIGGER_15"].values),float(angle))]
                                candlesDataBase_BigQuery_Signal_COPY.reset_index(inplace=True,drop=True)

                                #RSI 15 ESTE POR DEBAJO DE 50
                                candlesDataBase_BigQuery_Signal_COPY=candlesDataBase_BigQuery_Signal_COPY.loc[candlesDataBase_BigQuery_Signal_COPY.RSI_15<=float(rsi_15)]
                                candlesDataBase_BigQuery_Signal_COPY.reset_index(inplace=True,drop=True)

                                #PENDIENTE 15 MACD > 0
                                candlesDataBase_BigQuery_Signal_COPY=candlesDataBase_BigQuery_Signal_COPY.loc[candlesDataBase_BigQuery_Signal_COPY.PENDIENTE_MACD_15>0]
                                candlesDataBase_BigQuery_Signal_COPY.reset_index(inplace=True,drop=True)

                                #RSI 30 ESTE POR DEBAJO DE 50
                                candlesDataBase_BigQuery_Signal_COPY=candlesDataBase_BigQuery_Signal_COPY.loc[candlesDataBase_BigQuery_Signal_COPY.RSI_30<=float(rsi_30)]
                                candlesDataBase_BigQuery_Signal_COPY.reset_index(inplace=True,drop=True)

                                #PENDIENTE 30 MACD > 0
                                candlesDataBase_BigQuery_Signal_COPY=candlesDataBase_BigQuery_Signal_COPY.loc[candlesDataBase_BigQuery_Signal_COPY.PENDIENTE_MACD_30>0]
                                candlesDataBase_BigQuery_Signal_COPY.reset_index(inplace=True,drop=True)

                                #RSI 60 ESTE POR DEBAJO DE 50
                                candlesDataBase_BigQuery_Signal_COPY=candlesDataBase_BigQuery_Signal_COPY.loc[candlesDataBase_BigQuery_Signal_COPY.RSI_60<=float(rsi_60)]
                                candlesDataBase_BigQuery_Signal_COPY.reset_index(inplace=True,drop=True)

                                #PENDIENTE 60 MACD > 0
                                candlesDataBase_BigQuery_Signal_COPY=candlesDataBase_BigQuery_Signal_COPY.loc[candlesDataBase_BigQuery_Signal_COPY.PENDIENTE_MACD_60>0]
                                candlesDataBase_BigQuery_Signal_COPY.reset_index(inplace=True,drop=True)

                                #

                                #APPENDEAR COMPRA O NO COMRPA
                                listaComprar=list(candlesDataBase_BigQuery_Signal_COPY.TIME.values)
                                for i in list(test_sample.index.values):
                                    if test_sample.at[i,"TIME"] in listaComprar:
                                        test_sample.at[i,"PRED"]="SI"
                                    else:
                                        test_sample.at[i,"PRED"]="NO"

                                #_____COPIAR BASE DE DATOS
                                test_sample_copy=test_sample.copy()

                                #_____SOBREESCRIBIR VARIABLE CLOSE CON VALORES REALES
                                for back in range(0,len(test_sample_copy)):
                                    test_sample_copy.at[back,"CLOSE"]=backtestingDataFrame.loc[backtestingDataFrame.TIME==test_sample_copy.at[back,"TIME"]]["CLOSE"].values[0]

                                #_____CORREGIR PREDICTION
                                predict=test_sample_copy["PRED"]

                                #_____INICIAR PARAMETROS
                                date=test_sample_copy["TIME"]
                                close_price=test_sample_copy["CLOSE"]

                                #_____CREAR LIBRO DE TRADEO
                                tradingBook=pd.DataFrame(columns=["DATE_BUY","DATE_SELL","NET_BUY_PRICE","NET_SELL_PRICE","STOP_LOSS","NET_UTILITY"])

                                #_____PARAMETROS DE INICIO
                                j=0
                                ejecuto_compra=0

                                #_____LOOP SEMI-INFINITO
                                while j<=len(close_price)-2 :

                                    #_____TRIGGER DE COMPRA
                                    if predict[j] == "SI" :
                                        buy_authorization = "SI"
                                    else:
                                        buy_authorization = "NO"

                                    #_____SI TRIGGER DE COMPRA SE DISPARA
                                    if buy_authorization == "SI":

                                        #_____AGREGAR DATOS A LIBRO DE TRADEO + SUMAR UN POCO MAS DE FEES PARA AJUSTAR A VALORES REALES
                                        time_buy=date[j]
                                        index_row=len(tradingBook)
                                        tradingBook.at[index_row,"DATE_BUY"]=time_buy
                                        tradingBook.at[index_row,"DATE_SELL"]=""
                                        tradingBook.at[index_row,"NET_BUY_PRICE"]=close_price[j]*(1+0.001)

                                        #_____LOOP INFINITO HASTA QUE SE EJECUTE VENTA
                                        ejecuto_venta=0 
                                        while ejecuto_venta==0 :

                                            #_____SI PRECIO COMIENZA A CAER                
                                            if (ejecuto_venta==0) and j<=(len(close_price)-2):

                                                #_____ACTUALIZAR TIEMPO TRANSCURRIDO
                                                j+=1

                                                #_____ACTUALIZAR MARGEN DE VENTA + SUMAR UN POCO MAS DE FEES PARA AJUSTAR A VALORES REALES
                                                margen_actual= ((close_price[j]*(1-0.001))/tradingBook.at[index_row,"NET_BUY_PRICE"])-1

                                                #_____VENDER SI ESTOY ARRIBA DE MARGEN DE UTLIDAD
                                                if (ejecuto_venta==0) and (margen_actual >= takeprofit):

                                                    #_____EJECUTAR ORDEN DE VENTA
                                                    ejecuto_venta=1

                                                    #_____ACTUALIZAR BASE DE DATOS TRADINGBOOK_1
                                                    tradingBook.at[index_row,"DATE_SELL"]=date[j]
                                                    tradingBook.at[index_row,"NET_SELL_PRICE"]=(close_price[j]*(1-0.001))
                                                    tradingBook.at[index_row,"STOP_LOSS"]=0
                                                    tradingBook.at[index_row,"NET_UTILITY"]=(tradingBook.at[index_row,"NET_SELL_PRICE"]/tradingBook.at[index_row,"NET_BUY_PRICE"])-1

                                                #_____SI PRECIO ESTA EN REGION DE STOPLOSS
                                                elif (ejecuto_venta==0) and ((close_price[j] < tradingBook.at[index_row,"NET_BUY_PRICE"]*(1-stoploss+0.001))):

                                                    #_____EJECUTAR ORDEN DE VENTA
                                                    ejecuto_venta=1

                                                    #_____ACTUALIZAR BASE DE DATOS TRADINGBOOK_1
                                                    tradingBook.at[index_row,"DATE_SELL"]=date[j]
                                                    tradingBook.at[index_row,"NET_SELL_PRICE"]=close_price[j]
                                                    tradingBook.at[index_row,"STOP_LOSS"]=1
                                                    tradingBook.at[index_row,"NET_UTILITY"]=((close_price[j]*(1-0.001))/tradingBook.at[index_row,"NET_BUY_PRICE"])-1

                                            else:
                                                ejecuto_venta=1

                                    else:
                                        #_____ACTUALIZAR TIEMPO TRANSCURRIDO
                                        j+=1

                                #_____AGREGAR DATOS A BASE DE DATOS DE UTILIDAD
                                index_row=len(utility_df)
                                utility_df.at[index_row,"TP"]=takeprofit
                                utility_df.at[index_row,"SL"]=stoploss
                                utility_df.at[index_row,"DISTANCE"]=distance[0]
                                utility_df.at[index_row,"ANGLE"]=angle
                                utility_df.at[index_row,"RSI_15"]=rsi_15
                                utility_df.at[index_row,"RSI_30"]=rsi_30
                                utility_df.at[index_row,"RSI_60"]=rsi_60
                                utility_df.at[index_row,"NET_UTILITY"]=sum(tradingBook.dropna().NET_UTILITY)
                                utility_df.at[index_row,"LOSSES"]=sum(tradingBook.STOP_LOSS)
                                utility_df.at[index_row,"WINNINGS"]=len(tradingBook.loc[tradingBook.STOP_LOSS==0])
                                
    return utility_df

#_____

#FUNCION QUE CREA PORTAFOLIO SEGUN LINEA DE EFICIENCIA DE PARETO
def is_pareto_efficient(costs, return_mask = True):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :param return_mask: True to return a mask
    :return: An array of indices of pareto-efficient points.
        If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    """
    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs>costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    if return_mask:
        is_efficient_mask = np.zeros(n_points, dtype = bool)
        is_efficient_mask[is_efficient] = True
        return is_efficient_mask
    else:
        return is_efficient
    
#_____

#FUNCION QUE RETORNA MEJOR CONFIGURACION SEGUN PARETO
def getPareto(backtesting_n):
    
    global backtesting_3
    
    updateConfig("semiManual.ini")
    
    #_____PORTAFOLIO JERARQUICO
    Pareto_df=backtesting_n.copy()
    Pareto_df=Pareto_df.loc[(Pareto_df.LOSSES<=int(get_config("PARAMETERS_BACKTESTING","MAX_LOSSES")))&(Pareto_df.WINNINGS>len(candlesDataBase_BigQuery_Signal)*int(fractal)/60/24/7)].sort_values(by='NET_UTILITY', ascending=False)
    Pareto_df.reset_index(inplace=True,drop=True)
    Pareto_df=Pareto_df[["SL","RSI_15","RSI_30","RSI_60","NET_UTILITY","LOSSES","WINNINGS"]]

    Pareto_df["LOSSES"]=Pareto_df["LOSSES"]*(-1)
    Pareto_df["SL"]=Pareto_df["SL"]*(-1)
    Pareto_df["RSI_15"]=Pareto_df["RSI_15"]*(-1)
    Pareto_df["RSI_30"]=Pareto_df["RSI_30"]*(-1)
    Pareto_df["RSI_60"]=Pareto_df["RSI_60"]*(-1)

    Pareto_df.reset_index(inplace=True,drop=True)
    Efficient_market=is_pareto_efficient(np.array(Pareto_df), return_mask = False)
    Pareto_df=Pareto_df.loc[list(Efficient_market),:]

    Pareto_df["LOSSES"]=Pareto_df["LOSSES"]*(-1)
    Pareto_df["SL"]=Pareto_df["SL"]*(-1)
    Pareto_df["RSI_15"]=Pareto_df["RSI_15"]*(-1)
    Pareto_df["RSI_30"]=Pareto_df["RSI_30"]*(-1)
    Pareto_df["RSI_60"]=Pareto_df["RSI_60"]*(-1)
    Pareto_df.reset_index(inplace=True,drop=True)

    if len(Pareto_df)>0:
        backtesting_3=backtesting_n.copy()
        for i in list(Pareto_df.columns):
            backtesting_3=backtesting_3.loc[backtesting_3[i]==Pareto_df.at[0,i]]
        backtesting_3.reset_index(inplace=True, drop=True)
    
        if len(backtesting_3)>1:
            backtesting_3=backtesting_3.loc[backtesting_3.ANGLE==max(backtesting_3.ANGLE.values)]
            backtesting_3.reset_index(inplace=True, drop=True)

        if len(backtesting_3)>1:
            backtesting_3=backtesting_3.loc[backtesting_3.DISTANCE==max(backtesting_3.DISTANCE.values)]
            backtesting_3.reset_index(inplace=True, drop=True)

        if len(backtesting_3)>1:
            backtesting_3=backtesting_3[0:1]
            backtesting_3.reset_index(inplace=True, drop=True)

        backtesting_3.at[0,"NET_UTILITY"]=round_decimals_down(backtesting_3.at[0,"NET_UTILITY"],3)
    
    else:
        backtesting_3=Pareto_df.copy()
        
    return backtesting_3
    
#_____
    
#FINCION QUE FILTRA MERCADOS POTENCIALES POR VOLUMEN TRANSADO
def marketList():
    
    #_____CONEXION BINANCE
    binance = ccxt.binance()

    #_____ACTUALIZAR ARCHIVO CONFIG PARA MERCADOS
    updateConfig("semiManual.ini")

    #_____CREAR LISTA DE MERCADOS A EVALUAR
    mercados_Portafolio_Bruto=[]
    mercados=ast.literal_eval(get_config("DATABASES","MARKETS"))

    #_____LISTA VOLUMEN MERCADOS
    marketZeroVolumes_Prop=[]
    marketAverVolumes_List=[]

    #_____LISTA SAFE DE MERCADOS
    safeList=[]
    
    #_____LISTA TOP DE MERCADOS CMC
    cmc_Top_Market_List=topCoinMarketCap()

    #_____RECORRER MERCADOS
    for mercado in mercados:

        #_____DESCARGAR BASE DE DATOS OLCV
        #candlesDataBase_Binance = binance.fetch_ohlcv(mercado+"/USDT","1m",limit=1000)
        #candlesDataBase_Binance = candlesDataBase_Binance[:-1]

        #_____PROPORCION MERCADO MUERTO
        #contador_cero=0
        #contador_aver=0
        #for i in list(range(0,len(candlesDataBase_Binance))):
        #    if candlesDataBase_Binance[i][5] == 0:
        #        contador_cero=contador_cero+1
        #    contador_aver=contador_aver+candlesDataBase_Binance[i][5]

        #marketZeroVolumes_Prop.append(contador_cero/len(candlesDataBase_Binance))
        #marketAverVolumes_List.append(contador_aver/len(candlesDataBase_Binance))

        #_____ACTUALIZAR SAFE LIST
        #if ((contador_cero/len(candlesDataBase_Binance)) == 0):
        #    safeList.append(mercado)
        
        #_____SLEEP
        #time.sleep(2)
        
        #_____AGEREGAR MERCADO SI ESTA EN TOP CMC
        if mercado in cmc_Top_Market_List:
            safeList.append(mercado)
        

    #_____NUEVA LISTA DE MERCADOS
    return safeList

#_____

#FUNCION QUE GUARDA PDF COMO IMAGEN
def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

#_____

#TWITTER CONNECTION
def OAuth():
   
    global auth
    
    updateConfig("semiManual.ini")
    
    API_KEY=get_config("TWITTER","API_KEY")
    API_SECRET_KEY=get_config("TWITTER","API_SECRET_KEY")
    ACCESS_TOKEN=get_config("TWITTER","ACCESS_TOKEN")
    ACCESS_TOKEN_SECRET=get_config("TWITTER","ACCESS_TOKEN_SECRET")
   
    
    try:
        auth = tw.OAuthHandler(API_KEY, API_SECRET_KEY)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth

    except Exception as e:
        return None
    
#_____

#FUNCION QUE DEVUELVE EL TOG X DE MERCADOS
def topCoinMarketCap():
    
    updateConfig("semiManual.ini")
    
    API_KEY_CAP=get_config("COINMARKETCAP","API_KEY_CAP")
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit': get_config("MARKETS","TOP_CMC"),
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': API_KEY_CAP,
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)

    response = pd.DataFrame(data["data"])
    response=list(response.symbol.values)

    list_Avoid=["USDT","USDC","BUSD","UST","TUSD","USDN","LUSD","HUSD","OUSD","GUSD","USDX","vUSDC,""CUSD","SUSD","vBUSD,""MUSD","vUSDT,""USDK","USDJ"]

    for i in list_Avoid:
        try:
            response.remove(i)
        except:
            pass

    return response

# DATA

In [3]:
# #_____CCXT
# binance = ccxt.binance()

# #_____GOOGLE CLOUD CONECTION
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/PREDICTION/bigQueryAccess.json"

# #_____UPDATE CONFIG PREDICTION
# updateConfig("semiManual.ini")

# fractal="15"
# market_model="LUNA/USDT"

# _____CARGAR BASE DE DATOS
# myData = binance.fetch_ohlcv(market_model,fractal+"m",limit=1000)
# candlesDataBase_BigQuery =pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

# for i in range(0,len(myData)):
#     candlesDataBase_BigQuery.at[i,"ID"]=myData[i][0]
#     candlesDataBase_BigQuery.at[i,"TIME"]=datetime.fromtimestamp(myData[i][0]/1000.0)
#     candlesDataBase_BigQuery.at[i,"MARKET"]=market_model
#     candlesDataBase_BigQuery.at[i,"OPEN"]=myData[i][1]
#     candlesDataBase_BigQuery.at[i,"HIGH"]=myData[i][2]
#     candlesDataBase_BigQuery.at[i,"LOW"]=myData[i][3]
#     candlesDataBase_BigQuery.at[i,"CLOSE"]=myData[i][4]
#     candlesDataBase_BigQuery.at[i,"VOLUME"]=myData[i][5]

# # #_____CARGAR BASE DE DATOS
# # market_model=market_model+"_1M"
# # candlesDataBase_BigQuery=downloadDataBaseBigQuery(market_model,time=fractal,rows=int(get_config("PARAMETERS","ROWS_DOWNLOAD")))

# #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
# backtestingDataFrame=candlesDataBase_BigQuery.copy()

# #_____CREAR VARIABLES INPUT
# candlesDataBase_BigQuery=variableCreation(candlesDataBase_BigQuery)

# #_____CALCULAR PARA VELA DE 15
# for i in range(1,max(candlesDataBase_BigQuery.index.values)+1):
#     candlesDataBase_BigQuery.at[i,"RSI_15"]=candlesDataBase_BigQuery.at[i,"RSI"]
#     candlesDataBase_BigQuery.at[i,"HIST_MACD_VS_TRIGGER_15"]=(candlesDataBase_BigQuery.at[i,"MACD"]-candlesDataBase_BigQuery.at[i,"MACD_TRIGGER"])
#     candlesDataBase_BigQuery.at[i,"PENDIENTE_MACD_15"]=(candlesDataBase_BigQuery.at[i,"MACD"]-candlesDataBase_BigQuery.at[i-1,"MACD"])/int(fractal)
#     candlesDataBase_BigQuery.at[i,"PENDIENTE_MACD_TRIGGER_15"]=(candlesDataBase_BigQuery.at[i,"MACD_TRIGGER"]-candlesDataBase_BigQuery.at[i-1,"MACD_TRIGGER"])/int(fractal)
#     candlesDataBase_BigQuery.at[i,"ANGLE_MACD_VS_TRIGGER_15"]= np.arctan((candlesDataBase_BigQuery.at[i,"PENDIENTE_MACD_15"]-candlesDataBase_BigQuery.at[i,"PENDIENTE_MACD_TRIGGER_15"])/(1+candlesDataBase_BigQuery.at[i,"PENDIENTE_MACD_15"]*candlesDataBase_BigQuery.at[i,"PENDIENTE_MACD_TRIGGER_15"]))

# #_____CALCULAR PARA VELA DE 30
# fractal="30"

# #_____CARGAR BASE DE DATOS
# myData = binance.fetch_ohlcv(market_model,fractal+"m",limit=1000)
# candlesDataBase_BigQuery_2 =pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

# for i in range(0,len(myData)):
#     candlesDataBase_BigQuery_2.at[i,"ID"]=myData[i][0]
#     candlesDataBase_BigQuery_2.at[i,"TIME"]=datetime.fromtimestamp(myData[i][0]/1000.0)
#     candlesDataBase_BigQuery_2.at[i,"MARKET"]=market_model
#     candlesDataBase_BigQuery_2.at[i,"OPEN"]=myData[i][1]
#     candlesDataBase_BigQuery_2.at[i,"HIGH"]=myData[i][2]
#     candlesDataBase_BigQuery_2.at[i,"LOW"]=myData[i][3]
#     candlesDataBase_BigQuery_2.at[i,"CLOSE"]=myData[i][4]
#     candlesDataBase_BigQuery_2.at[i,"VOLUME"]=myData[i][5]

# # #_____CARGAR BASE DE DATOS
# # candlesDataBase_BigQuery_2=downloadDataBaseBigQuery(market_model,time=fractal,rows=int(get_config("PARAMETERS","ROWS_DOWNLOAD")))

# #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
# backtestingDataFrame_2=candlesDataBase_BigQuery_2.copy()

# #_____CREAR VARIABLES INPUT
# candlesDataBase_BigQuery_2=variableCreation(candlesDataBase_BigQuery_2)

# #_____CALCULAR PARA VELA DE 15
# for i in range(1,max(candlesDataBase_BigQuery_2.index.values)+1):
#     candlesDataBase_BigQuery_2.at[i,"RSI_30"]=candlesDataBase_BigQuery_2.at[i,"RSI"]
#     candlesDataBase_BigQuery_2.at[i,"PENDIENTE_MACD_30"]=(candlesDataBase_BigQuery_2.at[i,"MACD"]-candlesDataBase_BigQuery_2.at[i-1,"MACD"])/int(fractal)
    
# #_____CALCULAR PARA VELA DE 60
# fractal="1h"

# #_____CARGAR BASE DE DATOS
# myData = binance.fetch_ohlcv(market_model,fractal,limit=1000)
# candlesDataBase_BigQuery_3 =pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

# for i in range(0,len(myData)):
#     candlesDataBase_BigQuery_3.at[i,"ID"]=myData[i][0]
#     candlesDataBase_BigQuery_3.at[i,"TIME"]=datetime.fromtimestamp(myData[i][0]/1000.0)
#     candlesDataBase_BigQuery_3.at[i,"MARKET"]=market_model
#     candlesDataBase_BigQuery_3.at[i,"OPEN"]=myData[i][1]
#     candlesDataBase_BigQuery_3.at[i,"HIGH"]=myData[i][2]
#     candlesDataBase_BigQuery_3.at[i,"LOW"]=myData[i][3]
#     candlesDataBase_BigQuery_3.at[i,"CLOSE"]=myData[i][4]
#     candlesDataBase_BigQuery_3.at[i,"VOLUME"]=myData[i][5]

# # #_____CARGAR BASE DE DATOS
# # fractal="60"
# # candlesDataBase_BigQuery_3=downloadDataBaseBigQuery(market_model,time=fractal,rows=int(get_config("PARAMETERS","ROWS_DOWNLOAD")))

# #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
# backtestingDataFrame_3=candlesDataBase_BigQuery_3.copy()

# #_____CREAR VARIABLES INPUT
# candlesDataBase_BigQuery_3=variableCreation(candlesDataBase_BigQuery_3)

# #_____CALCULAR PARA VELA DE 15
# for i in range(1,max(candlesDataBase_BigQuery_3.index.values)+1):
#     candlesDataBase_BigQuery_3.at[i,"RSI_60"]=candlesDataBase_BigQuery_3.at[i,"RSI"]
#     candlesDataBase_BigQuery_3.at[i,"PENDIENTE_MACD_60"]=(candlesDataBase_BigQuery_3.at[i,"MACD"]-candlesDataBase_BigQuery_3.at[i-1,"MACD"])/int(60)

# #_____UNIR BASES DE DATOS
# candlesDataBase_BigQuery=candlesDataBase_BigQuery[["TIME","MARKET","RSI_15","HIST_MACD_VS_TRIGGER_15","PENDIENTE_MACD_15","ANGLE_MACD_VS_TRIGGER_15"]]
# candlesDataBase_BigQuery_2=candlesDataBase_BigQuery_2[["TIME","RSI_30","PENDIENTE_MACD_30"]]
# candlesDataBase_BigQuery_3=candlesDataBase_BigQuery_3[["TIME","RSI_60","PENDIENTE_MACD_60"]]

# candlesDataBase_BigQuery_Signal=pd.merge(candlesDataBase_BigQuery, candlesDataBase_BigQuery_2, on='TIME',how="left")
# candlesDataBase_BigQuery_Signal=pd.merge(candlesDataBase_BigQuery_Signal, candlesDataBase_BigQuery_3, on='TIME',how="left")

# candlesDataBase_BigQuery_Signal.fillna(method='ffill',inplace=True)
# # candlesDataBase_BigQuery_Signal.dropna(inplace=True)
# candlesDataBase_BigQuery_Signal=candlesDataBase_BigQuery_Signal[26*4:]
# candlesDataBase_BigQuery_Signal.reset_index(drop=True,inplace=True)

GRAPH

In [4]:
# #_____PARAMETROS
# candlesDataBase_Graph=candlesDataBase_BigQuery_Signal.copy()
# predType="PRED"

# #_____AGREGAR VARIABLE CLOSE
# for i in range(0,len(candlesDataBase_Graph)):
#     candlesDataBase_Graph.at[i,"CLOSE"]=backtestingDataFrame.loc[backtestingDataFrame.TIME==candlesDataBase_Graph.at[i,"TIME"]]["CLOSE"].values[0]

# #_____AGREGAR COLORES
# candlesDataBase_Color=candlesDataBase_Graph.copy()
# candlesDataBase_Color=candlesDataBase_Color[[predType]]
# candlesDataBase_Color=np.where(candlesDataBase_Color[predType] =="SI", "red", "blue")

# #_____DAR TEXTO A EJES
# x_buy = candlesDataBase_Graph[["TIME"]].values
# y_buy = candlesDataBase_Graph[["CLOSE"]].values
# color_buy = candlesDataBase_Color

# #_____RESETEAR
# fig_buy = None

# #_____IMPRIMIR GRAFICA
# fig_buy = plt.figure(1, figsize=(20,10))
# ax_buy  = fig_buy.add_subplot(111)
# plot_colourline(x_buy,y_buy,color_buy)

# CHECKS

# RUN

In [ ]:
#_____OPENMARKETS
openMarkets_Long=[]
writePickleVariable(openMarkets_Long,"openMarkets_Long")
openMarkets_Short=[]
writePickleVariable(openMarkets_Short,"openMarkets_Short")

#_____TWITTED MARKETS
openMarkets_Short_Twitted=[]
writePickleVariable(openMarkets_Short_Twitted,"openMarkets_Short_Twitted")
openMarkets_Long_Twitted=[]
writePickleVariable(openMarkets_Long_Twitted,"openMarkets_Long_Twitted")

#_____LOOP INFINITO
while True:
    
    #_____TRY DE CONTINGENCIA
    try:
    
        #_____CONEXION BINANCE
        binance = ccxt.binance()

        #_____UPDATE CONFIG PREDICTION
        updateConfig("semiManual.ini")

        #_____MARKET LIST DE MERCADOS POTENCIALES
        marketsIterationList=marketList()
        random.shuffle(marketsIterationList)

        #_____FOR LOOP PARA ITERACION DE ALERTA DE MERCADOS
        for market in marketsIterationList:

            #_____PRINT WITNESS
            clear_output()
            market_model=market+"/USDT"
            print(market+"/USDT",str(datetime.now()-timedelta(hours=5)))

            #_____CALCULAR PARA VELA DE 4H
            fractal="4h"

            #_____CARGAR BASE DE DATOS
            myData = binance.fetch_ohlcv(market_model,fractal,limit=1000)
            candlesDataBase_BigQuery_4H =pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

            for i in range(0,len(myData)):
                candlesDataBase_BigQuery_4H.at[i,"ID"]=myData[i][0]
                candlesDataBase_BigQuery_4H.at[i,"TIME"]=datetime.fromtimestamp(myData[i][0]/1000.0)
                candlesDataBase_BigQuery_4H.at[i,"MARKET"]=market_model
                candlesDataBase_BigQuery_4H.at[i,"OPEN"]=myData[i][1]
                candlesDataBase_BigQuery_4H.at[i,"HIGH"]=myData[i][2]
                candlesDataBase_BigQuery_4H.at[i,"LOW"]=myData[i][3]
                candlesDataBase_BigQuery_4H.at[i,"CLOSE"]=myData[i][4]
                candlesDataBase_BigQuery_4H.at[i,"VOLUME"]=myData[i][5]

            #_____CREAR VARIABLES INPUT
            candlesDataBase_BigQuery_4H=variableCreation(candlesDataBase_BigQuery_4H)

            #_____CALCULAR PARA VELA DE 4H
            for i in range(1,max(candlesDataBase_BigQuery_4H.index.values)+1):
                candlesDataBase_BigQuery_4H.at[i,"HIST_MACD_VS_TRIGGER_4H"]=(candlesDataBase_BigQuery_4H.at[i,"MACD"]-candlesDataBase_BigQuery_4H.at[i,"MACD_TRIGGER"])

            #_____SI MERCADO SE ABRE PARA LONG
            openMarkets_Long_Twitted=readPickleVariable("openMarkets_Long_Twitted")
            if market in openMarkets_Long_Twitted:
                if candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-1,"HIST_MACD_VS_TRIGGER_4H"]>0:
                    openMarkets_Long_Twitted.remove(market)
                    writePickleVariable(openMarkets_Long_Twitted,"openMarkets_Long_Twitted")
            if market not in openMarkets_Long_Twitted:
                if candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-1,"HIST_MACD_VS_TRIGGER_4H"]<0:
                    if candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-2,"HIST_MACD_VS_TRIGGER_4H"]<candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-1,"HIST_MACD_VS_TRIGGER_4H"]:
                        if candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-3,"HIST_MACD_VS_TRIGGER_4H"]<candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-2,"HIST_MACD_VS_TRIGGER_4H"]:
                            openMarkets_Long=readPickleVariable("openMarkets_Long")
                            if market not in openMarkets_Long:
                                openMarkets_Long.append(market)
                                writePickleVariable(openMarkets_Long,"openMarkets_Long")

            #_____SI MERCADO SE ABRE PARA SHORT
            openMarkets_Short_Twitted=readPickleVariable("openMarkets_Short_Twitted")
            if market in openMarkets_Short_Twitted:
                if candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-1,"HIST_MACD_VS_TRIGGER_4H"]<0:
                    openMarkets_Short_Twitted.remove(market)
                    writePickleVariable(openMarkets_Short_Twitted,"openMarkets_Short_Twitted")
            if market not in openMarkets_Short_Twitted:
                if candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-1,"HIST_MACD_VS_TRIGGER_4H"]>0:
                    if candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-2,"HIST_MACD_VS_TRIGGER_4H"]>candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-1,"HIST_MACD_VS_TRIGGER_4H"]:
                        if candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-3,"HIST_MACD_VS_TRIGGER_4H"]>candlesDataBase_BigQuery_4H.at[len(candlesDataBase_BigQuery_4H)-2,"HIST_MACD_VS_TRIGGER_4H"]:
                            openMarkets_Short=readPickleVariable("openMarkets_Short")
                            if market not in openMarkets_Short:
                                openMarkets_Short.append(market)
                                writePickleVariable(openMarkets_Short,"openMarkets_Short")

            #_____SLEEP
            time.sleep(20)
            clear_output()
            
    except:
        
        #_____PRINT
        print("[[ERROR]]")
        time.sleep(300)